In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
# !pip install OpenAI
import json
import pandas as pd
import numpy as np
from ParserDB import ParserDB

from openai import OpenAI
import os
with open("./data/data_json/openai_token.txt", "r") as f:
    os.environ["OPENAI_API_KEY"] = f.read()

In [78]:
# openai structuded output
request_format = {
    "format": {
        "type": "json_schema",
        "name": "user_stances_batch",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "threads": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "threadId": {"type": "string"},
                            "users": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "user": {"type": "string"},
                                        "stance": {
                                            "type": "string",
                                            "enum": ["поддерживает", "не поддерживает", "невозможно сказать"]
                                        }
                                    },
                                    "required": ["user", "stance"],
                                    "additionalProperties": False
                                }
                            }
                        },
                        "required": ["threadId", "users"],
                        "additionalProperties": False
                    }
                }
            },
            "required": ["threads"],
            "additionalProperties": False
        }
    }
}

# prompts for GPT-model
request_text_chemtrails =\
"""Тебе дано 10 тредов комментариев на ютубе. \
Пользователи под видеороликами спорят о конспирологической теории химтрейлов, \
то есть о распылении химикатов самолётами. Противники теории считают, что это инверсионный след, а не химикаты. \
Ты должен для каждого пользователя каждого треда определить, поддерживает он теорию о химтрейлах, не поддерживает, \
или невозможно определить. Формат данных: @пользователь: текст комментария. Если в тексте комментария есть @@username, \
значит автор комментария отвечает какому-то юзеру. Для каждого пользователя каждого треда определи, \
поддерживает он теорию о химтрейлах, не поддерживает, или невозможно определить.\
В ответе имена пользователей начинай записывать с @, так же, как в тексте треда. \
Для каждого треда определяй независимо."""

request_text_newchron =\
"""Тебе дано 10 тредов комментариев на ютубе. \
Пользователи под видеороликами спорят о конспирологической теории "Новой хронологии" Фоменко. \
Это такая псевдонаучная программа пересмотра истории, оппонирующая "официальной истории". \
Ты должен для каждого пользователя каждого треда определить, поддерживает он новую хронологию, не поддерживает, \
или невозможно определить. Формат данных: @пользователь: текст комментария. Если в тексте комментария есть @@username, \
значит автор комментария отвечает какому-то юзеру. Для каждого пользователя каждого треда определи, \
поддерживает он новую хронологию, не поддерживает, или невозможно определить. \
В ответе имена пользователей начинай записывать с @, так же, как в тексте треда. \
Для каждого треда определяй независимо."""

request_text_flatearth =\
"""Тебе дано 10 тредов комментариев на ютубе. \
Пользователи под видеороликами спорят о конспирологической теории плоской Земли. \
Сторонники теории считают, что планета Земля является плоской/вогнутой/какой-то ещё, а не шарообразной, и спорят с "официальной наукой". \
Ты должен для каждого пользователя каждого треда определить, поддерживает он теорию плоской земли, не поддерживает, \
или невозможно определить. Формат данных: @пользователь: текст комментария. Если в тексте комментария есть @@username, \
значит автор комментария отвечает какому-то юзеру. Для каждого пользователя каждого треда определи, \
поддерживает он теорию плоской земли, не поддерживает, или невозможно определить. \
В ответе имена пользователей начинай записывать с @, так же, как в тексте треда. \
Для каждого треда определяй независимо."""

In [6]:
p = ParserDB(api_key_file="./data/data_json/api_token.txt", database="./data/data.db")
p.comments_to_csv("химтрейлы", "./data/comments_chemtrail.csv")
p.comments_to_csv("новая хронология", "./data/comments_newchron.csv")
p.comments_to_csv("теория плоской земли", "./data/comments_flatearth.csv")

In [82]:
def create_batches(comments_file, output_folder, prefix, request_text, request_format):
    REQUEST_SIZE = 10 # threads in request
    BATCH_SIZE = 15000 # requests in batch
    
    df = pd.read_csv(comments_file)
    df = df[df['text'].notna()].sort_values(by=['topLevelComment', 'publishedAt'])
    threads = df.groupby(['topLevelComment'])[["authorDisplayName", "text"]].apply(
        lambda g: '\n'.join(f"{row['authorDisplayName']}: {row['text']}" for _, row in g.iterrows())).reset_index(name='thread')
    batches = [threads[i:i+BATCH_SIZE] for i in range(0, len(threads), BATCH_SIZE)]
    for i, b in enumerate(batches):
        with open(f"{output_folder}/{prefix}_{i+1}.jsonl", "w", encoding="utf-8") as f:
            for request_index in range(0, len(b), REQUEST_SIZE):
                chunk = b.iloc[request_index:request_index + REQUEST_SIZE]
                combined_text = "\n\n".join(
                    f"Тред {row['topLevelComment']}:\n{row['thread']}" for _, row in chunk.iterrows()
                )
                request_id = f"{(BATCH_SIZE // REQUEST_SIZE) * i + request_index // REQUEST_SIZE + 1}"
                
                obj = {
                    "custom_id": request_id,
                    "method": "POST",
                    "url": "/v1/responses",
                    "body": {
                        "model": "gpt-4.1-mini",
                        "input": [
                            {"role": "system", "content": request_text},
                            {"role": "user", "content": combined_text}
                        ],
                        "text": request_format
                    }
                }
                f.write(json.dumps(obj, ensure_ascii=False) + "\n")

In [84]:
client = OpenAI()

def batch_to_api(input_file):
    batch_input_file = client.files.create(
        file=open(input_file, "rb"),
        purpose="batch"
    )
    batch_input_file_id = batch_input_file.id
    client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/responses",
        completion_window="24h",
        metadata={}
    )

In [79]:
create_batches("./data/comments_chemtrail.csv", "./data/openai_batches", "batch_chemtrails", request_text_chemtrails, request_format)

In [80]:
create_batches("./data/comments_newchron.csv", "./data/openai_batches", "batch_newchron", request_text_newchron, request_format)

In [83]:
create_batches("./data/comments_flatearth.csv", "./data/openai_batches", "batch_flatearth", request_text_flatearth, request_format)

In [85]:
batch_to_api("./data/openai_batches/batch_newchron_1.jsonl")

In [91]:
def stances_by_thread(data):
    user_stances = {} # {user: {supports: 3, ...}}
    
    with open(data, "r") as f:
        for l in f.readlines():
            response_text = json.loads(json.loads(l)["response"]["body"]["output"][0]["content"][0]["text"])
            for thread in response_text["threads"]:
                for user in thread["users"]:
                    username = user["user"]
                    userstance = user["stance"]
                    cur_user_stance = user_stances.get(username, {})
                    cur_user_stance[userstance] = cur_user_stance.get(userstance, 0) + 1
                    user_stances[username] = cur_user_stance
    return user_stances

def define_user_stance(stance_counts):
    if stance_counts.get("поддерживает", 0) == stance_counts.get("не поддерживает", 0):
        return "невозможно определить"
    return "поддерживает" if stance_counts.get("поддерживает", 0) > stance_counts.get("не поддерживает", 0) else "не поддерживает"

def make_graph(threads_by_videos, user_stances, folder_to_save):
    nodes = set()
    edges = set()
    
    reply_count = 0
    problem_count = 0
    for videoId, videoThreads, query in threads_by_videos:
        videoThreads = json.loads(videoThreads)
        users = set()
        thread_authors = dict() # {topLevelComment: author}
        nodes.add((videoId, "video", query))
        for thread in videoThreads:
            thread = json.loads(thread)
            for comment in thread:
                users.add(comment["authorDisplayName"])
                nodes.add((comment["authorDisplayName"], "user", "USER"))
                if comment["topLevelComment"] == comment["commentId"]:
                    thread_authors[comment["topLevelComment"]] = comment["authorDisplayName"]
                
        for thread in videoThreads:
            thread = json.loads(thread)
            thread_author = thread_authors[thread[0]["topLevelComment"]]
            for comment in thread:
                if comment["authorDisplayName"] == thread_author:
                    edges.add((comment["authorDisplayName"], videoId))
                elif "@" not in comment["text"]:
                    edges.add((comment["authorDisplayName"], thread_author))
                else:
                    reply_to = None
                    for user in users:
                        if user in comment["text"]:
                            reply_to = user
                            break
                    if reply_to is not None:
                        edges.add((comment["authorDisplayName"], reply_to))
                    else:
                        problem_count += 1

    nodes = [dict(id=n[0], label=n[0], nodeType=n[1], query=n[2], stance=define_user_stance(user_stances.get(n[0], {}))) for n in nodes]
    edges = [dict(source=e[0], target=e[1]) for e in edges]
    
    nodes_df = pd.DataFrame(nodes)
    nodes_df["size"] = np.where(nodes_df["nodeType"] == "video", 20, 1)
    nodes_df.to_csv(f"{folder_to_save}/nodes.csv", index=False, encoding="utf-8")
    pd.DataFrame(edges).to_csv(f"{folder_to_save}/edges.csv", index=False, encoding="utf-8")

In [105]:
# EI-index: (E - I) / (E + I)
# E - external connections (responses to opponents); I - internal connections (responses to group-members)
def calculate_ei(nodes, edges):
    merged = pd.merge(
        edges,
        nodes.rename(columns={"label": "source", "stance": "source_stance"}),
        on="source"
    )[["target", "source", "source_stance"]]
    merged = pd.merge(
        merged,
        nodes.rename(columns={"label": "target", "stance": "target_stance"}),
        on="target"
    )[["target", "source", "source_stance", "target_stance"]]
    merged = merged[(merged["source_stance"] != "невозможно определить") & (merged["target_stance"] != "невозможно определить")]
    
    reply_counts = merged.groupby(by=["source_stance", "target_stance"]).size()
    
    I_support = reply_counts.loc["поддерживает"]["поддерживает"]
    I_oppose = reply_counts.loc["не поддерживает"]["не поддерживает"]
    E_support = reply_counts.loc["поддерживает"]["не поддерживает"]
    E_oppose = reply_counts.loc["не поддерживает"]["поддерживает"]

    print(I_support, I_oppose, E_support, E_oppose)
    
    EI_support = (E_support - I_support) / (E_support + I_support)
    EI_oppose = (E_oppose - I_oppose) / (E_oppose + I_oppose)
    return EI_support, EI_oppose

In [101]:
chemtrails_threads = p.get_threads("химтрейлы")
chemtrails_stances = stances_by_thread("./data/openai_responses/batch_chemtrails_output.jsonl")

In [102]:
make_graph(chemtrails_threads, chemtrails_stances, "data/gephi_chem_stances")

In [103]:
chemtrails_nodes = pd.read_csv("./data/gephi_chem_stances/nodes.csv")
chemtrails_edges = pd.read_csv("./data/gephi_chem_stances/edges.csv")

In [95]:
newchron_threads = p.get_threads("новая хронология")
newchron_stances = stances_by_thread("./data/openai_responses/batch_newchron_1_output.jsonl")
make_graph(newchron_threads, newchron_stances, "data/gephi_newchron_stances")

In [97]:
newchron_nodes = pd.read_csv("./data/gephi_newchron_stances/nodes.csv")
newchron_edges = pd.read_csv("./data/gephi_newchron_stances/edges.csv")

In [106]:
calculate_ei(chemtrails_nodes, chemtrails_edges)

4909 1030 1236 2450


(-0.5977217249796583, 0.40804597701149425)

In [107]:
calculate_ei(newchron_nodes, newchron_edges)

986 964 728 1306


(-0.15052508751458576, 0.15066079295154186)